In [1]:
import sys
# 移动默认路径
sys.path.append("../")

In [2]:
import os
import re
import jieba
import pandas as pd

In [3]:
from utils.path_config import *

### 导入csv文件

In [4]:
# 通过正则表达式去除停用词
def segment_line(line):
    line = re.sub(
            "[a-zA-Z0-9]|[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】《》“”！，。？、~@#￥%……&*（）]+|题目", '',line)
    tokens = jieba.cut(line, cut_all=False)
    return " ".join(tokens)

In [5]:
def get_segment(subject_section, subject_path):
    iter_ = 0
    for label, path in zip(subject_section, subject_path):
        df = pd.read_csv(path)
        df['item'] = df['item'].apply(lambda x:segment_line(x))
        df['label'] = label
        # 第一次循环初始化dataset_df
        if iter_ == 0:
            dataset_df = df
            iter_ += 1
        else:
            dataset_df=pd.concat([dataset_df,df])
    return dataset_df

In [7]:
%%time
history_df = get_segment(history_section, history_path_all)
politics_df = get_segment(politics_section, politics_path_all)
geography_df = get_segment(geography_section, geography_path_all)
biological_df = get_segment(biological_section, biological_path_all)

CPU times: user 33.1 s, sys: 24 ms, total: 33.1 s
Wall time: 33.1 s


In [8]:
dataset_df = pd.concat([history_df, politics_df, geography_df, biological_df])

In [9]:
dataset_df.head()

,web-scraper-order,web-scraper-start-url,item,label
0,1566523436-2497,https://study.baidu.com/tiku,据 左传 记载 春秋 后期 鲁国 大夫 季孙氏 的 家臣 阳虎 独掌 权柄 后 标榜 要 替...,古代史
1,1566523436-2506,https://study.baidu.com/tiku,秦始皇 统一 六国后 创制 了 一套 御玺 如 任命 国家 官员 则 封印 皇帝 之玺 ； ...,古代史
2,1566523436-2153,https://study.baidu.com/tiku,北宋 加强 中央集权 的 主要 措施 有 ① 把 主要 将领 的 兵权 收归 中央 ② 派 ...,古代史
3,1566523436-2328,https://study.baidu.com/tiku,商朝人 崇信 各种 鬼神 把 占卜 祭祀 作为 与 神灵 沟通 的 手段 负责 通神 事务 ...,古代史
4,1566523436-1914,https://study.baidu.com/tiku,公元 年 北宋 政府 在 江淮地区 设置 了 包括 盐业 管理 以及 控制 对 茶叶 销售 ...,古代史


In [11]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26243 entries, 0 to 1039
Data columns (total 4 columns):
web-scraper-order        26243 non-null object
web-scraper-start-url    26243 non-null object
item                     26243 non-null object
label                    26243 non-null object
dtypes: object(4)
memory usage: 1.6+ MB


In [12]:
corpus=dataset_df['item']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer

### TDIDF

In [15]:
%%time
# 设置2500个特征值，忽略词频小于5的词。
vectorizer = TfidfVectorizer(max_features=2500,min_df=5)
X = vectorizer.fit_transform(corpus)

CPU times: user 2.51 s, sys: 24 ms, total: 2.53 s
Wall time: 2.53 s


In [16]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test=train_test_split(X.toarray(),dataset_df['label'],test_size=0.2,random_state=42)

## sklearn
[naive bayes](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.naive_bayes)

In [19]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB,BernoulliNB
from sklearn.metrics import classification_report

# Naive Bayes

In [20]:
%%time
# 高斯朴素贝叶斯
clf = GaussianNB()
clf.fit(X_train, y_train)
Gauss_pred = clf.predict(X_test)

CPU times: user 1.07 s, sys: 264 ms, total: 1.33 s
Wall time: 1.33 s


In [21]:
print(classification_report(y_test, Gauss_pred))

              precision    recall  f1-score   support

       人口与城市       0.77      0.84      0.80       325
   公民道德与伦理常识       0.83      0.82      0.82       375
       分子与细胞       0.92      0.89      0.90       544
     区域可持续发展       0.38      0.25      0.30        24
         古代史       0.58      0.82      0.68       193
       地球与地图       0.27      0.57      0.36        86
      宇宙中的地球       0.98      0.83      0.90       730
        时事政治       1.00      0.17      0.29        12
         现代史       0.49      0.23      0.31       478
    现代生物技术专题       0.33      0.69      0.44       178
   生产活动与地域联系       0.77      0.72      0.74       279
    生活中的法律常识       0.89      0.39      0.54        41
      生物技术实践       0.55      0.33      0.41       360
     生物科学与社会       0.67      0.10      0.18       812
      科学思维常识       0.74      0.33      0.46        42
    科学社会主义常识       0.71      0.75      0.73       114
       稳态与环境       0.20      0.86      0.33       203
       经济学常识       0.68    

In [22]:
%%time
# 多项式朴素贝叶斯
clf = MultinomialNB()
clf.fit(X_train, y_train)
Multi_pred = clf.predict(X_test)

CPU times: user 327 ms, sys: 43.6 ms, total: 370 ms
Wall time: 145 ms


In [23]:
print(classification_report(y_test, Multi_pred))

              precision    recall  f1-score   support

       人口与城市       0.92      0.97      0.94       325
   公民道德与伦理常识       0.84      0.98      0.91       375
       分子与细胞       0.92      0.94      0.93       544
     区域可持续发展       0.00      0.00      0.00        24
         古代史       0.96      0.85      0.90       193
       地球与地图       0.96      0.56      0.71        86
      宇宙中的地球       0.96      0.99      0.98       730
        时事政治       0.00      0.00      0.00        12
         现代史       0.71      0.83      0.77       478
    现代生物技术专题       0.38      0.42      0.40       178
   生产活动与地域联系       0.85      0.93      0.89       279
    生活中的法律常识       1.00      0.12      0.22        41
      生物技术实践       0.66      0.69      0.67       360
     生物科学与社会       0.80      0.89      0.84       812
      科学思维常识       1.00      0.74      0.85        42
    科学社会主义常识       0.96      0.96      0.96       114
       稳态与环境       0.79      0.23      0.35       203
       经济学常识       0.99    

/home/abner/anaconda3/envs/nlp02/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
